In [31]:
import sympy as sym
import numpy as np
#import pylab as pl
from IPython.display import display, Math, Latex
sym.init_printing(use_unicode=True)
import sympy.utilities.autowrap as symauto

epss,Dels,nus,ta,tb,pa,pb,kappa,kappb,kas,kbs =\
sym.symbols('epsilon Delta nus ta tb pa pb kappaa kappab kas kbs',real=True)
kapp1a,kapp1b,Las,Lbs = \
sym.symbols('kappa1a kappa1b Las Lbs',real=True)

#Functions for evaluation
Ema = sym.exp(sym.I*(-nus*ta+pa))
Emb = sym.exp(sym.I*(-nus*tb+pb))
Epa = sym.exp(sym.I*(nus*ta+pa))
Epb = sym.exp(sym.I*(nus*tb+pb))
dma = kappa-sym.I*(nus-Dels) + kas*Epa         #d_{-,a}(\nu)
dmb = kappb-sym.I*(nus+Dels) + kbs*Epb         #d_{-,b}(\nu)
dpa = kappa-sym.I*(nus+Dels) + kas*sym.conjugate(Ema)   #d_{+,a}(\nu)
dpb = kappb-sym.I*(nus-Dels) + kbs*sym.conjugate(Emb)   #d_{+,b}(\nu)
Lab = sym.Abs(epss)**2 - dpa*dmb          #\Lambda_{ab}(\nu)
Lba = sym.Abs(epss)**2 - dpb*dma          #\Lambda_{ba}(\nu)

#Expectation value of the outgoing photon number
nom_ba = (kappa+kas*sym.cos(pa-nus*ta)-kapp1a*Las)*(kappb+kbs*sym.cos(pb+nus*tb))
nom_ab = (kappb+kbs*sym.cos(pb+nus*tb)-kapp1b*Lbs)*(kappa+kas*sym.cos(pa-nus*ta))
I_pexp  = 4*epss**2*(nom_ba/(Lab*sym.conjugate(Lab))+nom_ab/(Lba*sym.conjugate(Lba)))
I_mexp  = 4*epss**2*(nom_ba/(Lab*sym.conjugate(Lab))-nom_ab/(Lab*sym.conjugate(Lab)))
#display(sym.simplify(sym.expand(I_mexp,complex=True)))
diffp   = sym.Rational(1,2)*(sym.diff(I_pexp,pa)-sym.diff(I_pexp,pb))
diffm   = sym.Rational(1,2)*(sym.diff(I_mexp,pa)-sym.diff(I_mexp,pb))
#display(sym.re(diffm))
#display(diffp)
derivp = sym.lambdify((nus,epss,Dels,kappa,kappb,kapp1a,kapp1b,kas,kbs,ta,tb,pa,pb,Las,Lbs), diffp, modules="numpy")
#derivm = sym.lambdify((nus,epss,Dels,kappa,kappb,kapp1a,kapp1b,kas,kbs,ta,tb,pa,pb,Las,Lbs), diffm, modules="numpy")
#num    = sym.lambdify((nu,eps,Del,kapa,kapb,kap1a,kap1b,ka,kb,ta,tb,phia,phib,La,Lb), I_pexp, modules="numpy")

In [5]:
import numpy as np
#%matplotlib inline
import matplotlib.pyplot as plt
import scipy.integrate as integ
from matplotlib.colors import LogNorm
from IPython.core.debugger import Tracer
#Tracer()() #this one triggers the debugger

def sens(x,delta,kapa,kapb,ra,rb,kta,ktb,phia,phib,La,Lb,nus,nups):
# Function to determine the phase sensitivity of the setup
    import time
    start  = time.clock()

    ## Parameters from inputs ##
    kapa  = kapa*2*np.pi
    kapb  = kapb*2*np.pi
    La    = La/100.
    Lb    = Lb/100.
    phia  = phia*np.pi
    phib  = phib*np.pi
    taua  = kta/kapa
    taub  = ktb/kapa
    Del   = delta*kapa
    eps   = x*kapa
    kap1a = kapa * ra                         #\kappa_{1,a} (coupling on the right hand side in mode a)
    kap1b = kapb * rb                         #\kappa_{1,b} (coupling on the right hand side in mode b)
    kap2a = kapa * (1-ra)                     #\kappa_{2,a} (coupling on the left hand side in mode a)
    kap2b = kapb * (1-rb)                     #\kappa_{2,b} (coupling on the left hand side in mode b)

    ka = 2*np.sqrt(ra*(1-ra)*(1-La))*kapa    #k_a (feedback strength in mode a)
    kb = 2*np.sqrt(rb*(1-rb)*(1-Lb))*kapb    #k_b (feedback strength in mode b)
    
    ### EXPRESSIONS ###
    def Ei(nu,t,phi):
        return np.exp(1j*(nu*t+phi))
    def d(nu,t,phi,kap,Del,k): #dpa if Del&phi, dma if -Del&-phi, dmb if Del&-phi, dpb if -Del&phi
        return kap - 1j*(nu+Del) + k*Ei(nu,t,-phi)
    def Lab(nu):
        return abs(eps)**2 - d(nu,taua,phia,kapa,Del,ka)*d(nu,taub,-phib,kapb,Del,kb)
    def Lba(nu):
        return abs(eps)**2 - d(nu,taub,phib,kapb,-Del,kb)*d(nu,taua,-phia,kapa,-Del,ka)
    def Ia(nu,kap1,kap2): #alpha if quantities for a, 1 if kap1=kap1
        return np.sqrt(2*kap1) + np.sqrt(2*kap2*(1-La))*Ei(nu,taua,phia)
    def Ib(nu,kap1,kap2): #alpha if quantities for a, 1 if kap1=kap1
        return np.sqrt(2*kap1) + np.sqrt(2*kap2*(1-Lb))*Ei(nu,taub,phib)

    def Da(nu): #Da if p is for b
        return np.sqrt(1-La)*Lba(nu)*Ei(nu,taua,phia) + d(nu,taub,phib,kapb,-Del,kb)*\
    Ia(nu,kap1a,kap2a)*Ia(nu,kap2a,kap1a)
    def Db(nu): #Da if p is for b
        return np.sqrt(1-Lb)*Lab(nu)*Ei(nu,taub,phib) + d(nu,taua,phia,kapa,Del,ka)*\
    Ib(nu,kap1b,kap2b)*Ib(nu,kap2b,kap1b)

    def Ea(nu): # Da if p is for b
        return np.sqrt(La)*(np.sqrt(2*kap2a)*d(nu,taub,phib,kapb,-Del,kb)*Ia(nu,kap2a,kap1a) + Lba(nu))
    def Eb(nu): # Da if p is for b
        return np.sqrt(Lb)*(np.sqrt(2*kap2b)*d(nu,taua,phia,kapa,Del,ka)*Ib(nu,kap2b,kap1b) + Lab(nu))

    def U1ab(nu,nup): # for U1ab p means b
        return np.abs(Ia(-nu,kap2a,kap1a))**2*(np.abs(Ib(nu,kap1b,kap2b))**2+2*kap2b*Lb)*\
    (np.abs(Da(nup))**2+np.abs(Ea(nup))**2)
    def U1ba(nu,nup): # for U1ab p means b
        return np.abs(Ib(-nu,kap2b,kap1b))**2*(np.abs(Ia(nu,kap1a,kap2a))**2+2*kap2a*La)*\
    (np.abs(Db(nup))**2+np.abs(Eb(nup))**2)
    def U2ab(nu,nup): # for U2ab p means b
        return np.conj(Ia(-nu,kap2a,kap1a))*Ib(-nup,kap2b,kap1b)*\
    (Ib(nu,kap1b,kap2b)*np.conj(Db(nu))+np.sqrt(2*kap2b*Lb)*np.conj(Eb(nu)))*\
    (np.conj(Ia(nup,kap1a,kap2a))*Da(nup)+np.sqrt(2*kap2a*La)*Ea(nup))
    def U2ba(nu,nup): # for U2ab p means b
        return np.conj(Ib(-nu,kap2b,kap1b))*Ia(-nup,kap2a,kap1a)*\
    (Ia(nu,kap1a,kap2a)*np.conj(Da(nu))+np.sqrt(2*kap2a*La)*np.conj(Ea(nu)))*\
    (np.conj(Ib(nup,kap1b,kap2b))*Db(nup)+np.sqrt(2*kap2b*Lb)*Eb(nup))
    
    def DelIsq1ab(nu,nup):
        return np.abs(eps)**2*U1ab(nu,nup)/np.abs(Lab(nu))**2/np.abs(Lba(nup))**2
    def DelIsq1ba(nu,nup):
        return np.abs(eps)**2*U1ba(nu,nup)/np.abs(Lba(nu))**2/np.abs(Lab(nup))**2
    def DelIsq2ab(nu,nup): # for ab p means b
        return np.abs(eps)**2*U2ab(nu,nup)/np.abs(Lab(nu))**2/np.abs(Lba(nup))**2
    def DelIsq2ba(nu,nup): # for ab p means b
        return np.abs(eps)**2*U2ba(nu,nup)/np.abs(Lba(nu))**2/np.abs(Lab(nup))**2

    def DelIsq(nu,nup):
        return DelIsq1ab(nu,nup)+DelIsq1ba(nu,nup)+(DelIsq2ab(nu,nup)+DelIsq2ba(nu,nup))
#    def denom(nu,nup):
        return np.real(derivp(2*np.pi*nu,eps,Del,kapa,kapb,kap1a,kap1b,ka,kb,taua,taub,phia,phib,La,Lb)*\
                 derivp(2*np.pi*nup,eps,Del,kapa,kapb,kap1a,kap1b,ka,kb,taua,taub,phia,phib,La,Lb))
    
    nu,nup = np.meshgrid(nus,nups)
    plt.figure(1)
    plt.contourf(nu,nup,DelIsq(2*np.pi*nu,2*np.pi*nup),400,cmap=plt.cm.jet,norm = LogNorm())
    plt.colorbar()
 #   plt.figure(2)
 #   plt.contourf(nu,nup,denom(nu,nup),400,cmap=plt.cm.jet,norm = LogNorm())
 #   plt.figure(3)
  #  plt.plot(nus,derivp(2*np.pi*nus,eps,Del,kapa,kapb,kap1a,kap1b,ka,kb,taua,taub,phia,phib,La,Lb))
    plt.show()
    
nu = np.linspace(-100,100,501)
nup = np.linspace(-100,100,501)
sens(0.75,0,10,10,0.5,0.5,1.8833,1.8833,10,10,0,0,nu,nup)
plt.contourf(nu,nup,derivp(nu,0.75,0,10,10,5,5,10,10,ta,tb,pa,pb,Las,Lbs))

/home/nnem614/Documents/anaconda3/envs/py35/lib/python3.5/site-packages/numpy/ma/core.py:2717: ComplexWarning: Casting complex values to real discards the imaginary part
  order=order, subok=True, ndmin=ndmin)


NameError: name 'derivp' is not defined

In [10]:
sens(0.75,0,10,10,0.5,0.5,1.8833,1.8833,0,0,0,0,nu,nup)

/home/nnem614/Documents/anaconda3/envs/py35/lib/python3.5/site-packages/numpy/ma/core.py:2717: ComplexWarning: Casting complex values to real discards the imaginary part
  order=order, subok=True, ndmin=ndmin)


In [188]:
import numpy as np
#%matplotlib inline
import matplotlib.pyplot as plt
import scipy.integrate as integ
from matplotlib.colors import LogNorm
from IPython.core.debugger import Tracer
#Tracer()() #this one triggers the debugger

def sens(x,delta,kapa,kapb,ra,rb,kta,ktb,phia,phib,La,Lb,nus,nups):
# Function to determine the phase sensitivity of the setup
    import time
    start  = time.clock()

    ## Parameters from inputs ##
    kapa  = kapa*2*np.pi
    kapb  = kapb*2*np.pi
    La    = La/100.
    Lb    = Lb/100.
    phia  = phia*np.pi
    phib  = phib*np.pi
    taua  = kta/kapa
    taub  = ktb/kapa
    Del   = delta*kapa
    eps   = x*kapa
    kap1a = kapa * ra                         #\kappa_{1,a} (coupling on the right hand side in mode a)
    kap1b = kapb * rb                         #\kappa_{1,b} (coupling on the right hand side in mode b)
    kap2a = kapa * (1-ra)                     #\kappa_{2,a} (coupling on the left hand side in mode a)
    kap2b = kapb * (1-rb)                     #\kappa_{2,b} (coupling on the left hand side in mode b)

    ka = 2*np.sqrt(ra*(1-ra)*(1-La))*kapa    #k_a (feedback strength in mode a)
    kb = 2*np.sqrt(rb*(1-rb)*(1-Lb))*kapb    #k_b (feedback strength in mode b)
    
    ### EXPRESSIONS ###
    def Ei(nu,t,phi):
        return np.exp(1j*(nu*t+phi))
    def d(nu,t,phi,kap,Del,k): #dpa if Del&phi, dma if -Del&-phi, dmb if Del&-phi, dpb if -Del&phi
        return kap - 1j*(nu+Del) + k*Ei(nu,t,-phi)
    def Lab(nu):
        return abs(eps)**2 - d(nu,taua,phia,kapa,Del,ka)*d(nu,taub,-phib,kapb,Del,kb)
    def Lba(nu):
        return abs(eps)**2 - d(nu,taub,phib,kapb,-Del,kb)*d(nu,taua,-phia,kapa,-Del,ka)
    def fa(nu,kap1,kap2): #alpha if quantities for a, 1 if kap1=kap1
        return np.sqrt(2*kap1) + np.sqrt(2*kap2*(1-La))*Ei(nu,taua,phia)
    def fb(nu,kap1,kap2): #alpha if quantities for a, 1 if kap1=kap1
        return np.sqrt(2*kap1) + np.sqrt(2*kap2*(1-Lb))*Ei(nu,taub,phib)

    def Da(nu): #Da if p is for b
        return np.sqrt(1-La)*Lba(nu)*Ei(nu,taua,phia) + d(nu,taub,phib,kapb,-Del,kb)*\
    fa(nu,kap1a,kap2a)*fa(nu,kap2a,kap1a)
    def Db(nu): #Da if p is for b
        return np.sqrt(1-Lb)*Lab(nu)*Ei(nu,taub,phib) + d(nu,taua,phia,kapa,Del,ka)*\
    fb(nu,kap1b,kap2b)*fb(nu,kap2b,kap1b)

    def Ea(nu): # Da if p is for b
        return np.sqrt(La)*(np.sqrt(2*kap2a)*d(nu,taub,phib,kapb,-Del,kb)*fa(nu,kap2a,kap1a) + Lba(nu))
    def Eb(nu): # Da if p is for b
        return np.sqrt(Lb)*(np.sqrt(2*kap2b)*d(nu,taua,phia,kapa,Del,ka)*fb(nu,kap2b,kap1b) + Lab(nu))

    def U1ab(nup): # for U1ab p means b
        return Da(nup)*(np.conj(Da(nup))+np.conj(fa(nup,kap1a,kap2a))*fb(-nup,kap2b,kap1b)*np.abs(eps))
    def U1ba(nup): # for U1ab p means b
        return Db(nup)*(np.conj(Db(nup))+np.conj(fb(nup,kap1b,kap2b))*fa(-nup,kap2a,kap1a)*np.abs(eps))
    def U2ab(nup): # for U2ab p means b
        return Ea(nup)*(np.conj(Ea(nup))+np.sqrt(2*kap2a*La)*fb(-nup,kap2b,kap1b)*np.abs(eps))
    def U2ba(nup): # for U2ab p means b
        return Eb(nup)*(np.conj(Eb(nup))+np.sqrt(2*kap2b*Lb)*fa(-nup,kap2a,kap1a)*np.abs(eps))
    def U3ab(nu): # for U1ab p means b
        return np.abs(eps)*np.conj(fa(-nu,kap2a,kap1a))*\
    fb(nu,kap1b,kap2b)*(fa(-nu,kap2a,kap1a)*np.conj(fb(nu,kap1b,kap2b))*np.abs(eps)+np.conj(Db(nu)))
    def U3ba(nu): # for U1ab p means b
        return np.abs(eps)*np.conj(fb(-nu,kap2b,kap1b))*\
    fb(nu,kap1a,kap2a)*(fa(-nu,kap2b,kap1b)*np.conj(fa(nu,kap1a,kap2a))*np.abs(eps)+np.conj(Da(nu)))
    def U4ab(nu): # for U1ab p means b
        return np.abs(eps)*np.conj(fa(-nu,kap2a,kap1a))*\
    np.sqrt(2*kap2b*Lb)*(fa(-nu,kap2a,kap1a)*np.sqrt(2*kap2b*Lb)*np.abs(eps)+np.conj(Eb(nu)))
    def U4ba(nu): # for U1ab p means b
        return np.abs(eps)*np.conj(fb(-nu,kap2b,kap1b))*\
    np.sqrt(2*kap2a*La)*(fb(-nu,kap2b,kap1b)*np.sqrt(2*kap2a*La)*np.abs(eps)+np.conj(Ea(nu)))
    
    def F1(nup):
        return np.real(1/np.abs(Lba(2*np.pi*nup))**2*(U1ab(2*np.pi*nup)+U2ab(2*np.pi*nup)))
    def F2(nu):
        return np.real(1/np.abs(Lab(2*np.pi*nu))**2*(U3ab(2*np.pi*nu)+U4ab(2*np.pi*nu)))
    def F3(nup):
        return np.real(1/np.abs(Lab(2*np.pi*nup))**2*(U1ba(2*np.pi*nup)+U2ba(2*np.pi*nup)))
    def F4(nu):
        return np.real(1/np.abs(Lba(2*np.pi*nu))**2*(U3ba(2*np.pi*nu)+U4ba(2*np.pi*nu)))
    
    N=len(nus)
    #nus[int((N-1)/2)]=0
    plt.figure(1)
    plt.plot(nus[int((N-1)/2):(N+1)],F1(nus[int((N-1)/2):(N+1)]),label = "F_1")
    plt.plot(nus[0:int((N+1)/2)],F2(nus[0:int((N+1)/2)]),label = "F_2")
#    plt.xlim(-20,20)
    plt.plot(nus[0:int((N+1)/2)],F3(nus[0:int((N+1)/2)]),label = "F_3")
    plt.plot(nus[int((N-1)/2):(N+1)],F4(nus[int((N-1)/2):(N+1)]),label = "F_4")
    plt.legend()
    plt.grid(True)
    
    F1_int = integ.quad(F1,0.0,10000)
    F2_int = integ.quad(F2,-10000,-0.0)
    F3_int = integ.quad(F3,-10000,-0.0)
    F4_int = integ.quad(F4,0.0,10000)
#    F1_int = integ.quad(F1,0,np.inf)
#    F2_int = integ.quad(F2,-np.inf,0)
#    F3_int = integ.quad(F3,-np.inf,0)
#    F4_int = integ.quad(F4,0,np.inf)

    def uncert1(nu,nup):
        return 1/np.abs(Lba(nup))**2*(U1ab(nup)+U2ab(nup))*1/np.abs(Lab(nu))**2*(U3ab(nu)+U4ab(nu))
    def uncert2(nu,nup):
        return 1/np.abs(Lab(nup))**2*(U1ba(nup)+U2ba(nup))*1/np.abs(Lba(nu))**2*(U3ba(nu)+U4ba(nu))
       
    nu1,nup1 = np.meshgrid(nus[int((N-1)/2):(N+1)],nups[0:int((N+1)/2)])
    nu2,nup2 = np.meshgrid(nus[0:int((N+1)/2)],nups[int((N-1)/2):(N+1)])
    plt.figure(2)
    plt.contourf(nu2,nup2,np.real(uncert1(2*np.pi*nu2,2*np.pi*nup2)),400,cmap=plt.cm.jet,norm = LogNorm())
    plt.contourf(nu1,nup1,np.real(uncert2(2*np.pi*nu1,2*np.pi*nup1)),400,cmap=plt.cm.jet,norm = LogNorm())
    plt.colorbar()
 #   plt.figure(2)
 #   plt.contourf(nu,nup,denom(nu,nup),400,cmap=plt.cm.jet,norm = LogNorm())
 #   plt.figure(3)
  #  plt.plot(nus,derivp(2*np.pi*nus,eps,Del,kapa,kapb,kap1a,kap1b,ka,kb,taua,taub,phia,phib,La,Lb))
#    plt.show()
    print(F1_int[0]*F2_int[0]+F3_int[0]*F4_int[0])
nu = np.linspace(-100,100,501)
nup = np.linspace(-100,100,501)

#sens(0.75,0,10,10,0.5,0.5,1.8833,1.8833,0,0.0,5,5,nu,nup)
sens(0.5,0,10,10,0.933,0.933,0,0,1,1,5,5,nu,nup)
#sens(1,5,10,10,1,1,0,0,0,0,0,0,nu,nup)
#plt.contourf(nu,nup,derivp(nu,0.75,0,10,10,5,5,10,10,ta,tb,pa,pb,Las,Lbs))


12275008.978694368


In [64]:
a=4
if a!=4:
    print(a)

In [91]:
48172745.16213238-23636716.585297097
7938612.123284881

24536028.576835286